In [346]:
import pandas as pd
import math

In [347]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [510]:
filename = "project3_dataset2.txt"

In [511]:
data_set = pd.read_csv(filename,sep = '\t',header = None,nrows = 1)
columns = data_set.columns.tolist()
cols_to_use = columns[:len(columns)]
default_class_label = pd.read_csv(filename,sep = '\t',header = None,usecols=columns[len(columns)-1 : ])
data_set = pd.read_csv(filename,sep = '\t',header = None,usecols=cols_to_use)

In [512]:
data_set.head(3)

,0,1,2,3,4,5,6,7,8,9
0,132,6.20,6.47,36.21,Present,62,30.77,14.14,45,0
1,123,0.05,4.61,13.69,Absent,51,23.23,2.78,16,0
2,128,0.50,3.70,12.81,Present,66,21.25,22.73,28,0


In [513]:
len(data_set)

462

### Handling categorical values

In [514]:
def prepare_data(dataframe):
    dataframe[4] = dataframe[4].astype('category')
    dataframe[4] = dataframe[4].cat.codes
    #dataframe = pd.DataFrame(preprocessing.normalize(dataframe))
    return dataframe
    

### Cross-validation

In [515]:
def split_train_test(k,dataframe,fold_no):
    length = int(len(dataframe)/10)
    test = pd.DataFrame(dataframe,index=range(fold_no*length,fold_no*length + length))
    train = dataframe.loc[~dataframe.index.isin(test.index)]
    return train,test   

In [516]:
data_set.head(3)

,0,1,2,3,4,5,6,7,8,9
0,132,6.20,6.47,36.21,Present,62,30.77,14.14,45,0
1,123,0.05,4.61,13.69,Absent,51,23.23,2.78,16,0
2,128,0.50,3.70,12.81,Present,66,21.25,22.73,28,0


In [517]:
data_set = prepare_data(data_set)

In [518]:
data_set.head(3)

,0,1,2,3,4,5,6,7,8,9
0,132,6.20,6.47,36.21,1,62,30.77,14.14,45,0
1,123,0.05,4.61,13.69,0,51,23.23,2.78,16,0
2,128,0.50,3.70,12.81,1,66,21.25,22.73,28,0


In [519]:
data_set.shape[1]

10

### Pre-processing data

In [520]:
scaler = MinMaxScaler()
print(scaler.fit(data_set))

MinMaxScaler(copy=True, feature_range=(0, 1))


In [521]:
data_set.head(3)

,0,1,2,3,4,5,6,7,8,9
0,132,6.20,6.47,36.21,1,62,30.77,14.14,45,0
1,123,0.05,4.61,13.69,0,51,23.23,2.78,16,0
2,128,0.50,3.70,12.81,1,66,21.25,22.73,28,0


In [522]:
data_set = pd.DataFrame(scaler.transform(data_set))

In [523]:
from numpy import linalg as LA
from scipy.spatial import distance

In [524]:
def get_label(k,data_point,train):
    train_1 = train.iloc(axis = 0)[:,-1]
    label_0 = []
    label_1 = []
    data_point = data_point.drop([data_point.shape[1] - 1],axis=1)
    for i,val in train.iterrows():
        train_data_point = pd.DataFrame(val).transpose()
        train_data_point_1 = train_data_point.copy()
        train_data_point_1 = train_data_point_1.drop([train_data_point_1.shape[1] - 1],axis=1)
        dst = distance.euclidean(data_point, train_data_point_1)
        if(train_data_point.iloc[0][train_data_point.shape[1] - 1] == 0):
            label_0.append(dst)
        elif(train_data_point.iloc[0][train_data_point.shape[1] - 1] == 1):
            label_1.append(dst)
    sorted(label_0)
    sorted(label_1)
    count = 0
    flag_0 = 0
    flag_1 = 0
    score_0 = 0
    score_1 = 0
    while count <= k:
        if (label_0[flag_0] > label_1[flag_1]):
            flag_0 = flag_0 + 1
            score_0 = score_0 + (1/(label_0[flag_0]**2))
        elif (label_0[flag_0] < label_1[flag_1]):
            flag_1 = flag_1 + 1
            score_1 = score_1 + (1/(label_1[flag_1]**2))
        count = count + 1
    if (score_0 > score_1):
        return 0
    else:
        return 1         

### K-NN Algorithm

In [525]:
def K_NN(dataframe):
    num_classes = 2
    if ((num_classes % 2) == 0):
        i = 1
    else:
        i = 0
    for k in range(i,int(math.sqrt(data_set.shape[0])),2):
        if k==0:
            continue
        print("k-value ", k)
        for fold_no in range(10):
            train,test = split_train_test(k,dataframe,fold_no)
            acc = []
            pre = []
            rec = []
            f_mea = []
            true_positive = 0
            true_negative = 0
            false_negative = 0
            false_positive = 0
            for i,data_point in test.iterrows():
                data_point = pd.DataFrame(data_point).transpose()
                predicted_label = get_label(k,data_point,train)
                if ((predicted_label == 1) and (data_point.iloc[0][data_point.shape[1] - 1] == 1.0)):
                    true_positive = true_positive + 1
                elif ((predicted_label == 0) and (data_point.iloc[0][data_point.shape[1] - 1] == 1.0)):
                    false_negative = false_negative + 1
                elif ((predicted_label == 1) and (data_point.iloc[0][data_point.shape[1] - 1] == 0.0)):
                    false_positive = false_positive + 1
                elif ((predicted_label == 0) and (data_point.iloc[0][data_point.shape[1] - 1] == 0.0)):
                    true_negative = true_negative + 1
            accuracy = (true_positive + true_negative)/ (true_positive + false_negative + false_positive + true_negative)
            precision = true_positive/ (true_positive + false_positive)
            recall = true_positive/(true_positive + false_negative)
            f_measure = (2 * true_positive)/ ((2 * true_positive) + false_negative + false_positive)
            acc.append(accuracy)
            pre.append(precision)
            rec.append(recall)
            f_mea.append(f_measure)
        print("Average accuracy: ",sum(acc)/float(len(acc)))
        print("Average precision: ",sum(pre)/float(len(pre)))
        print("Average recall: ",sum(rec)/float(len(rec)))
        print("Average f_measure: ",sum(f_mea)/float(len(f_mea)))

In [526]:
K_NN(data_set)

k-value  1
Average accuracy:  0.6086956521739131
Average precision:  0.4166666666666667
Average recall:  0.7142857142857143
Average f_measure:  0.5263157894736842
k-value  3
Average accuracy:  0.5652173913043478
Average precision:  0.39285714285714285
Average recall:  0.7857142857142857
Average f_measure:  0.5238095238095238
k-value  5
Average accuracy:  0.4782608695652174
Average precision:  0.2727272727272727
Average recall:  0.42857142857142855
Average f_measure:  0.3333333333333333
k-value  7
Average accuracy:  0.41304347826086957
Average precision:  0.25925925925925924
Average recall:  0.5
Average f_measure:  0.34146341463414637
k-value  9
Average accuracy:  0.5217391304347826
Average precision:  0.3333333333333333
Average recall:  0.5714285714285714
Average f_measure:  0.42105263157894735
k-value  11
Average accuracy:  0.5434782608695652
Average precision:  0.34782608695652173
Average recall:  0.5714285714285714
Average f_measure:  0.43243243243243246
k-value  13
Average accuracy